In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
from sklearn import preprocessing

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv('AWSTFstack.csv', sep=',', header=None)
df = df.values

pts_target = pd.read_csv('output.csv', sep=',', header=None)

# Standardize the inputs
scaled_inputs = preprocessing.scale(df)
print(scaled_inputs)

samples_count = scaled_inputs.shape[0]

train_samples = int(0.8*samples_count)
test_samples = int(samples_count - train_samples)

train_inputs = scaled_inputs[:train_samples]
train_targets = pts_target[:train_samples]

test_inputs = scaled_inputs[train_samples:]
test_targets = pts_target[train_samples:]

[[ 0.44617255  1.02541799  0.15239653 ... -0.83934138 -0.56656537
  -0.35692569]
 [-0.91638412  1.02541799  1.70631619 ...  0.27779821 -0.62409691
  -0.35692569]
 [-1.73391812 -0.97521207  0.06569487 ... -0.83934138 -0.79669153
  -0.35692569]
 ...
 [-0.23510579 -0.97521207  0.06569487 ...  2.51207739 -0.56656537
  -0.35692569]
 [ 0.30991688  1.02541799  0.22197503 ... -0.83934138  1.50457012
  -0.35692569]
 [-1.05263978  1.02541799  0.2799571  ... -0.83934138  0.29640775
  -0.35692569]]


In [3]:
print("Training set: {}".format(train_inputs.shape))  # 1052 examples, 26 features
print("Testing set:  {}".format(test_inputs.shape))   # 263 examples, 26 features

Training set: (1052, 26)
Testing set:  (263, 26)


In [4]:
print(train_inputs[0])

[ 0.44617255  1.02541799  0.15239653  0.72567649  0.16851094 -0.99388244
 -0.96181435  1.35053238  1.1994434   1.20806276 -0.91668956  1.28651695
 -1.54278637 -1.29353661 -1.36863193 -0.49936737 -0.82444962 -0.73028071
 -0.36205584  2.40557078  1.15606111 -0.00945257 -0.49550798 -0.83934138
 -0.56656537 -0.35692569]


In [5]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu, 
                       input_shape=(train_inputs.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])
  #AdamOptimizer
  optimizer = tf.train.AdamOptimizer(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
  return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1728      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,953
Trainable params: 5,953
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Display training progress by printing a single dot for each completed epoch.
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 50

# Store training stats
history = model.fit(train_inputs, train_targets, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])


..................................................

In [7]:
import matplotlib.pyplot as plt


def plot_history(history):
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [1000$]')
  plt.plot(history.epoch, np.array(history.history['mean_absolute_error']), 
           label='Train Loss')
  plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']),
           label = 'Val loss')
  plt.legend()
  plt.ylim([0,5])

plot_history(history)

In [8]:
[loss, mae] = model.evaluate(test_inputs, test_targets, verbose=0)

print("Testing set Mean Abs Error: {:7.2f}".format(mae))

Testing set Mean Abs Error:    8.26


In [9]:
test_predictions = model.predict(test_inputs).flatten()

print(test_predictions)

[20.15827   24.260895  23.67173   21.844273  14.118347  22.095123
 18.958979  25.186401  16.7878    26.541386  23.38868   19.415363
 27.497906  24.148867  19.80375   27.356066  19.071104  23.001526
 16.669987  20.835     26.186905  27.340553  13.268856  22.910711
 22.165558  19.797646  22.554722  18.710808  21.255306  13.30571
 15.549063  18.185057  41.192066  21.970242  19.514557  20.26284
 20.287094  23.948631  20.92553   25.831205  18.137955  24.260143
 14.491581  17.793848  13.741529  18.675762  19.344332  17.993826
 12.717567  18.07016   15.152541  36.6462    15.781563  25.120096
 28.92471   27.219646  14.935747  21.8194    15.10667   20.101557
 20.994658  16.914654  21.59999   16.765347  16.477922  25.122686
 17.567057  18.882961  17.716215  22.034496  18.222015  31.872545
 17.595673  29.685648  14.757143  26.704468  19.365484  23.458988
 18.309715  12.9392605 21.59947   19.296947  12.844949  22.006031
 15.057063  30.977955  15.72364   25.397755  25.133179  30.63302
 23.435934  2

In [10]:
print(test_targets[0:262])

       0
1052   7
1053  15
1054   8
1055   8
1056  26
1057  13
1058   2
1059  18
1060  17
1061  32
1062  17
1063  35
1064  26
1065  29
1066   2
1067  31
1068  20
1069  10
1070  45
1071  35
1072  12
1073  20
1074  14
1075  20
1076  15
1077  16
1078   5
1079  10
1080   1
1081  14
...   ..
1284  17
1285  21
1286  15
1287  19
1288  12
1289  17
1290  17
1291  17
1292   2
1293  19
1294  15
1295  11
1296  12
1297   7
1298   0
1299  26
1300  18
1301  -1
1302  13
1303  28
1304   9
1305  16
1306  12
1307  14
1308  33
1309  19
1310  32
1311  17
1312   7
1313  23

[262 rows x 1 columns]


In [16]:
np.savetxt("predictions.csv", test_predictions, delimiter=",")

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'